In [4]:
import pandas as pd
import numpy as np
from keras.models import load_model
from keras.utils import pad_sequences
import ast  
from sklearn.metrics import mean_absolute_error, mean_squared_error

# load model
loaded_model = load_model('encoder_decoder_full_model_LSTM.h5')

df = pd.read_csv('easytoken.csv')
train_data = df['tokenized_id']

framenumber= 75
targetlength=int(framenumber*155*3)
pedvalue = 1.0


#perform preprocess to the input
train_data1 = train_data[0]
sample_list = ast.literal_eval(train_data1)
sample_list = np.array(sample_list)
sample_list = np.reshape(sample_list, (1, -1))
sample_list = pad_sequences(sample_list, maxlen=int(targetlength/framenumber), padding='post', truncating='post',dtype='float64',value=pedvalue)

input_data_expanded = np.tile(sample_list, (1, framenumber))

input_data_3d = input_data_expanded.reshape((1, framenumber, -1)) 

print(input_data_3d.shape)
#load the true label
train_labels = np.load("clip_5i9vRk-pAWY_0.pose_data.json_combined.npy")

train_labels = np.reshape(train_labels, (1, -1))
train_labels = pad_sequences(train_labels, maxlen=targetlength, padding='post', truncating='post',dtype='float64',value=pedvalue)
label_data_3d = train_labels.reshape((1, framenumber, -1))  
print(label_data_3d.shape)


# initializing the prediction
decoder_input_data = np.zeros((1, framenumber, 465)) 
decoder_input_data[:, 0, :] = label_data_3d[:, 0, :]


predictions = []
for i in range(framenumber):  
    # predict, if looking for providing a completely strange envirment for it, comment the next line and uncomment the line next tot he next line. 
    preds = loaded_model.predict([input_data_3d, label_data_3d])

    #The observed loss value for this situation is about 0.09

    #preds = loaded_model.predict([input_data_3d, decoder_input_data])
    # update
    decoder_input_data[:, i+1:i+2, :] = preds[:, i:i+1, :]
    
    predictions.append(preds[:, i, :])


predictions = np.concatenate(predictions, axis=1)




#preds = loaded_model.predict([input_data_3d, input_data_3d])


mae = mean_absolute_error(label_data_3d.reshape(-1), predictions.flatten())
print(mae)


#loaded_array1 = np.load('clip_5i9vRk-pAWY_6.pose_data.json_combined.npy')
#np.savetxt('truelabel.txt', loaded_array1, delimiter='\t')

#np.savetxt('output1.txt', predictions.ravel(), delimiter='\t')
np.save('prediction.npy', predictions)


(1, 75, 465)
(1, 75, 465)
1/1 [==============================] - 1s 882ms/step


ValueError: Found input variables with inconsistent numbers of samples: [34875, 465]